In [1]:
from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.operators.crossover.sbx import SBX
from pymoo.operators.mutation.pm import PM
from pymoo.util.remote import Remote

from pymoo.problems import get_problem
from pymoo.optimize import minimize
from pymoo.core.problem import Problem
from pymoo.operators.sampling.rnd import FloatRandomSampling,IntegerRandomSampling
from pymoo.operators.repair.rounding import RoundingRepair

import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
file = Remote.get_instance().load("examples", "portfolio_allocation.csv", to=None)
df = pd.read_csv(file, parse_dates=True, index_col="date") # aqui são os dados diários dos valores de 20 ações que vamos criar nosso portifólio

,GOOG,AAPL,FB,BABA,AMZN,GE,AMD,WMT,BAC,GM,T,UAA,SHLD,XOM,RRC,BBY,MA,PFE,JPM,SBUX
date,,,,,,,,,,,,,,,,,,,,
1989-12-29,NaN,0.117203,NaN,NaN,NaN,0.352438,3.9375,3.486070,1.752478,NaN,2.365775,NaN,NaN,1.766756,NaN,0.166287,NaN,0.110818,1.827968,NaN
1990-01-02,NaN,0.123853,NaN,NaN,NaN,0.364733,4.1250,3.660858,1.766686,NaN,2.398184,NaN,NaN,1.766756,NaN,0.173216,NaN,0.113209,1.835617,NaN
1990-01-03,NaN,0.124684,NaN,NaN,NaN,0.364050,4.0000,3.660858,1.780897,NaN,2.356516,NaN,NaN,1.749088,NaN,0.194001,NaN,0.113608,1.896803,NaN
1990-01-04,NaN,0.125100,NaN,NaN,NaN,0.362001,3.9375,3.641439,1.743005,NaN,2.403821,NaN,NaN,1.731422,NaN,0.190537,NaN,0.115402,1.904452,NaN
1990-01-05,NaN,0.125516,NaN,NaN,NaN,0.358586,3.8125,3.602595,1.705114,NaN,2.287973,NaN,NaN,1.722587,NaN,0.190537,NaN,0.114405,1.912100,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-04-05,1027.810059,172.800003,159.339996,172.570007,1451.750000,13.430000,10.0200,87.809998,30.320000,38.000000,35.632843,17.469999,2.97,76.019997,14.52,72.120003,175.550003,35.730000,111.879997,59.139999
2018-04-06,1007.039978,168.380005,157.199997,167.520004,1405.229980,13.060000,9.6100,86.690002,29.629999,37.680000,35.130001,16.980000,2.88,74.870003,13.97,70.489998,169.699997,35.169998,109.089996,58.340000
2018-04-09,1015.450012,170.050003,157.929993,169.869995,1406.079956,12.830000,9.5300,86.279999,29.870001,37.830002,35.169998,16.639999,2.82,74.870003,13.93,69.820000,170.339996,35.459999,110.400002,58.700001


In [ ]:
returns = df.pct_change().dropna(how="all")# calcula os retornos (pct_change) para todas as colunas, exceto NaN
print('Shapes of returns=',returns.shape)